In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sentence-transformers faiss-cpu
!pip install -q transformers accelerate sentence-transformers faiss-cpu
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [3]:
!pip install gradio
!pip install transformers sentence-transformers faiss-cpu gradio PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.2 MB/s eta 0:00:00


In [4]:
import os
import json
import pandas as pd
import numpy as np
import fitz
import faiss
import gradio as gr
import torch
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer,
    pipeline, DataCollatorWithPadding
)
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

from sentence_transformers import SentenceTransformer

# **RAG Implementation**

**Encoding the RAG corpus**

In [5]:
def load_pdf_text(path):
    doc = fitz.open(path)
    return [p.strip().replace('\n', ' ') for page in doc for p in page.get_text().split('\n\n') if len(p.strip()) > 40]

who_paragraphs = load_pdf_text("WHO.pdf")
ada_paragraphs = load_pdf_text("ADA.pdf")
rag_corpus = list(set(who_paragraphs + ada_paragraphs))

embedder = SentenceTransformer("all-MiniLM-L6-v2")
rag_embeddings = embedder.encode(rag_corpus, show_progress_bar=True)
index = faiss.IndexFlatL2(rag_embeddings.shape[1])
index.add(np.array(rag_embeddings))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

# **Chatbot UI and Multilingual Support**

In [6]:
base_model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-3b")
model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/finetuned_diabetes_bloomz_lora")
gen_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/finetuned_diabetes_bloomz_lora")
model.eval()

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.transformer.h.0.self_attention.query_key_value.lora_A.default.weight', 'base_model.model.transformer.h.0.self_attention.query_key_value.lora_B.default.weight', 'base_model.model.transformer.h.1.self_attention.query_key_value.lora_A.default.weight', 'base_model.model.transformer.h.1.self_attention.query_key_value.lora_B.default.weight', 'base_model.model.transformer.h.2.self_attention.query_key_value.lora_A.default.weight', 'base_model.model.transformer.h.2.self_attention.query_key_value.lora_B.default.weight', 'base_model.model.transformer.h.3.self_attention.query_key_value.lora_A.default.weight', 'base_model.model.transformer.h.3.self_attention.query_key_value.lora_B.default.weight', 'base_model.model.transformer.h.4.self_attention.query_key_value.lora_A.default.weight', 'base_model.model.transformer.h.4.self_attention.query_key_value

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 2560)
        (word_embeddings_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-29): 30 x BloomBlock(
            (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=7680, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=7680, bias=False)
                )
                (lora_em

In [38]:
translator_sp = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")
back_translator_sp = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")

translator_fr = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
back_translator_fr = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")

def generate_response(query, profile, language="English", history=[]):
    try:
        if language.lower() == "spanish":
            query = translator_sp(query)[0]['translation_text']
        elif language.lower() == "french":
            query = translator_fr(query)[0]['translation_text']

        query_vec = embedder.encode([query], convert_to_numpy=True)
        _, indices = index.search(np.array(query_vec), 5)
        context = "\n".join(rag_corpus[i] for i in indices[0][:2])
        prompt = (
            "You are a multilingual diabetes assistant. Respond briefly (1–2 lines), kindly, and based on CONTEXT and PATIENT INFO.\n"
            "Only give medically sound and safe advice. If unsure, recommend asking a doctor.\n"
            "---CONTEXT---\n"
            f"{context}\n"
            "---END CONTEXT---\n"
            f"Patient Info: {profile}\n"
            f"Question: {query}\n"
            "Answer:"
        )

        print(f"[DEBUG] Prompt: {prompt}")

        inputs = gen_tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=80,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                repetition_penalty=1.2,
                eos_token_id=gen_tokenizer.eos_token_id,
            )
        response = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract after "Answer:" keyword or fallback to the full generated text
        if "Answer:" in response:
            response = response.split("Answer:")[-1].strip()
        elif "Assistant:" in response:
            response = response.split("Assistant:")[-1].strip()
        else:
            # Fallback: remove everything before the user query
            if "Question:" in response:
                response = response.split("Question:")[-1].strip()

        # Remove repeated prompt if it's echoed back
        if response.startswith("You are a multilingual diabetes assistant"):
            response = response.replace("You are a multilingual diabetes assistant", "").strip()

        # Trim to one clean sentence
        response = response.split("\n")[0].strip()

        if language.lower() == "spanish":
            response = back_translator_sp(response)[0]['translation_text']
        elif language.lower() == "french":
            response = back_translator_fr(response)[0]['translation_text']

        return response

    except Exception as e:
        print(f"[\u274c Error] {e}")
        return f"\u274c Internal Error: {str(e)}"


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0


In [43]:
# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🧡 Multilingual Diabetes Chatbot")
    chatbot = gr.Chatbot()
    history_state = gr.State([])

    with gr.Row():
        with gr.Column():
            query = gr.Textbox(label="Health Query")
            age = gr.Number(label="Age")
            glucose = gr.Number(label="Glucose Level")
            symptoms = gr.Textbox(label="Symptoms")
            meds = gr.Textbox(label="Medications")
            lang = gr.Radio(["English", "Spanish", "French"], label="Language")
            submit = gr.Button("Submit")

    def chat_wrapper(query, age, glucose, symptoms, meds, lang, history):
        profile = f"Age: {age}, Glucose: {glucose}, Symptoms: {symptoms}, Medications: {meds}"
        response = generate_response(query, profile, lang, history)
        history.append((query, response))  # ✅ FIX: Gradio expects a tuple (user_msg, bot_msg)
        return history, history

    submit.click(chat_wrapper, inputs=[query, age, glucose, symptoms, meds, lang, history_state], outputs=[chatbot, history_state])

demo.launch()

<ipython-input-43-693dea186fa4>:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://21f8f34abcf68e8ff6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **Evaluation**

In [9]:
!pip install evaluate
!pip install rouge_score sacrebleu nltk bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [10]:
import evaluate
from evaluate import load
from bert_score import score as bertscore

In [34]:
# Load multi-turn data from saved jsonl file
eval_set = []
with open("/content/drive/MyDrive/diabetes_multiturn_finetune_data.jsonl", "r") as f:
    for line in f:
        eval_set.append(json.loads(line))

# Use only top 100 examples
eval_set = eval_set[:100]

# Generate predictions
outputs = []
for dialog in eval_set:
    user_msg = next((m["content"] for m in dialog["messages"] if m["role"] == "user"), "")
    true_answer = next((m["content"] for m in dialog["messages"] if m["role"] == "assistant"), "")
    input_text = f"User: {user_msg}\nAssistant:"

    inputs = gen_tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=50)

    generated = gen_tokenizer.decode(output_ids[0], skip_special_tokens=True).split("Assistant:")[-1].strip()
    outputs.append({"prediction": generated, "reference": true_answer})

# Save predictions for reuse
with open("diabetes_predictions.json", "w") as f:
    json.dump(outputs, f, indent=2)

# BLEU evaluation
bleu = evaluate.load("sacrebleu")
bleu_score = bleu.compute(
    predictions=[o["prediction"] for o in outputs],
    references=[[o["reference"]] for o in outputs]
)
print(f"\nBLEU Score: {bleu_score['score']:.2f}")

# BERTScore evaluation
bertscore = evaluate.load("bertscore")
bert_scores = bertscore.compute(
    predictions=[o["prediction"] for o in outputs],
    references=[o["reference"] for o in outputs],
    lang="en"
)
print(f"BERTScore (F1): {np.mean(bert_scores['f1']):.4f}")


BLEU Score: 0.00


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore (F1): 0.7873
